#### Тестовое задание на проект "Бот-помощник для поиска решений типовых проблем" JetBrains

Выполнил: Рыбин Иван, магистратура ИТМО JB "Software Engineering"

In [1]:
import requests

#### Можно сделать без токена

Лимит запросов 60 в час, нам нужен только один

#### Можно сделать через токен

Лимит запросов 5000 в час

1. Для начала необходимо создать токен аутентификации:

    https://docs.github.com/en/github/authenticating-to-github/creating-a-personal-access-token
    

2. Ввести github логин в поле ***user***

3. Ввести github токен из п.1 в поле ***token***

In [2]:
user = None
token = None

In [3]:
# запрос к API
def get_response(url, username=None, token=None):
    if not username:
        return requests.get(url=url)

    return requests.get(url=url, auth=(username, token))

In [4]:
# Сбор сообщений из json ответа
def get_messages_from_response(json_response):
    messages = []
    for commit in json_response:
        if 'commit' in commit and 'message' in commit['commit']:
            messages.append(commit['commit']['message'])
    return messages

# Сбор топ 30 слов с учетом регистра и без (пропускаем символы и цифры)
def get_top_30_words(messages):
    bad_signs = ['.', ',', '!', '/', '\\', "#", '$', '%', '^', '*', '(', ')', '@', '>', '<', ' ']
    all_words = [word for message in messages for word in message.split(' ') if word]
    all_words = [word.strip() for word in all_words if word not in bad_signs and not word.isdigit()]
    
    words_any_case = dict()
    words_lower_case = dict()
    
    for word in all_words:
        if word not in words_any_case:
            words_any_case[word] = 0   
        if word.lower() not in words_lower_case:
            words_lower_case[word.lower()] = 0
            
        words_any_case[word] += 1
        words_lower_case[word.lower()] += 1    
        
    words_any_case = {k: v for k, v in list(
                          sorted(words_any_case.items(), reverse=True, key=lambda item: item[1])
                      )[:30]
                     }
    words_lower_case = {k: v for k, v in list(
                          sorted(words_lower_case.items(), reverse=True, key=lambda item: item[1])
                      )[:30]
                     }
    return words_any_case, words_lower_case

def pretty_print(words):
    print("{: <15} {}".format("words", "count"))
    for word, cnt in words.items():
        print("{: <15} {}".format(word, cnt))

In [5]:
# берем первые 100 коммитов с проекта Kotlin
url = 'https://api.github.com/repos/JetBrains/kotlin/commits?per_page=100'
response = get_response(url, user, token)

In [6]:
content = response.json()
messages = get_messages_from_response(content)

In [7]:
print(f"Число коммитов: {len(content)}")
print(f"Число сообщений: {len(messages)}")

Число коммитов: 100
Число сообщений: 100


In [8]:
words_any_case, words_lower_case = get_top_30_words(messages)

In [9]:
print('Слова без учета регистра\n')
pretty_print(words_any_case)

Слова без учета регистра

words           count
for             37
in              29
to              27
[ULC]           26
the             19
of              17
and             15
Fixed           15
[FIR]           14
from            14
with            13
UL              13
Add             12
classes         12
not             10
on              9
private         9
type            9
FIR             9
is              8
Fix             8
return          8
Remove          8
invalid         8
usage           7
remove          7
JVM_IR:         7
IR:             6
types           6
annotations     6


In [10]:
print('Слова с приведением к нижнему регистру\n')
pretty_print(words_lower_case)

Слова с приведением к нижнему регистру

words           count
for             37
in              29
to              27
[ulc]           26
the             20
of              17
fixed           16
and             15
remove          15
add             14
[fir]           14
from            14
fix             13
with            13
ul              13
classes         12
not             10
fir             10
don't           9
on              9
private         9
type            9
is              8
return          8
invalid         8
usage           7
jvm_ir:         7
ir:             6
a               6
types           6
